# Advanced Risk Management – Assignment 2

**Deadline**:  14 March 2022, 18.00h.

| |Name |Student number|Email|
|:-|:----|:-------------|:----|
|1.|  |        |     |
|2.|  |        |     |
|3.|  |        |     |

****Hand in the following via Canvas****:
* Your notebook.
* A (printed) pdf version of your notebook. In Google Colab, this is most conveniently done in the Chrome browser, and then using the **`File` -> `Print`** menu option; you may have to print in landscape mode to make sure that everything appears in the pdf.
* Files must adhere to the following naming format to be considered eligible: 'Assignment1_STUDENTNUMBER1_STUDENTNUMBER2_STUDENTNUMBER3.ipynb' and 'Assignment1_STUDENTNUMBER1_STUDENTNUMBER2_STUDENTNUMBER3.pdf'. The files and the order of student numbers in the files names must be exactly identical for each member in a team. T


**Notes**:
* The assignment is part of the examination, so the usual rules regarding plagiarism and fraud apply.
* Before submitting your work, click on **`Runtime`-> `Restart and run all ...`** and verify that your notebook produces the desired results and does not error.

**Declaration of Originality**: We, whose names are given under 1., 2., and 3., above declare that these solutions are solely our own work, and that we have not made these solutions available to any other student.

## Introduction
The file `data_AEX_FTSE.xlsx` contains daily log returns of the AEX and FTSE index returns for the period 5 January 2000 - 12 March 2020. The log return of asset $i$ on day $t$ is defined as $R_{i,t} = 100 \cdot \log(\frac{S_{i,t}}{S_{i,t-1}})$, with $S_{i,t}$ the price of asset $i$ on date $t$.

 In this assignment you are asked to calculate multi-period Value-at-Risk forecasts through Monte Carlo simulation (MCS). First, we obtain the VaR for the AEX index. Subsequently, we employ copula methods to obtain the VaR for a portfolio of two assets.

## Question 1: Load, clean, and model data [20 pts.]
First, install and import the relevant libraries:

In [ ]:
!pip install arch             # uncomment for installing the arch package
import numpy as np
import os
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns

Import the data in a DataFrame. The data is already transformed using methods that were used in Assignment 1 as well. Uncomment and adapt the lines necessary to mount the drive and change the path.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/...'    # change path to your working directory
# os.chdir(path)

df = pd.read_excel('data_AEX_FTSE.xlsx') 
df = df.set_index(['Date']) # set 'Date' column as index

R_AEX = df['R_AEX']
R_FTSE = df['R_FTSE']

df.tail()

Suppose you are a risk manager and it is currently just after closing time of the trading day 12 March 2020, i.e. we are at the end of period $t=T$, with $T$ indicating 12 March 2020. You are asked to calculate the 10-day VaR on the AEX index, i.e. $VaR^p_{T+1:T+K}$, with horizon $K=10$ trading days. After some thought you decide to model the index with a GARCH model, and use Monte Carlo Simulation to obtain the 10-day VaR.

Estimate a GARCH(1,1) model, with constant mean return and standardized $t$ distributed shocks, for the AEX index returns. 

Plot the volatility series and analyze the shocks. Also obtain the one-day-ahead volatility forecast for 13 March 2021. Discuss your results. Can you attribute certain economic events to your observations for 12 March 2020?

Notes: 

* By specifying `dist = ` in the `arch_model` function we can change the distribution of the shocks. See the documentation at https://arch.readthedocs.io.
* GARCH model residuals can be obtain from the `.resid` attribute belonging to the fitted model (which is generated by the `.fit()` method). Conditional volatilities can be obtained similarly from the `.conditional_volatility` attribute. 
* The 1-day conditional variance forecast (note: variance instead of volatility) can be obtained as `x.forecast().variance['h.1']['2020-03-12']`, where `x` is the fitted model object, and the dates index refers to the date the forecast is made (so here we generate a forecast at the end of 12 March and forecast one-day ahead for the date 13 March.)

Discussion of results:

## Question 2: Monte Carlo Simulation of 10-day ahead VaR [25 pts.]
We use Monte Carlo simulation to obtain the 10-day ahead Value-at-Risk forecast at $p=0.01$, 10-days ahead of 12 March 2020. 

A 'code skeleton' is included below, which you should complete. Implement steps A to F:

* A: Replace `x` with the fitted GARCH model object generated by the `.fit()` method in the previous question.
* B: For date $T+k$, $k=1$ we know the conditional variance forecast. Replace `xx` with the value you obtained for 13 March 2020 in the previous question.
* C: For dates $T+k$, $k > 1$ we use the GARCH model specification. Replace `xxx` with the GARCH equation: $\sigma_{T+k}^2 = \omega + \alpha (\sigma_{T+k-1} z_{T+k-1})^2 + \beta \sigma_{T+k-1}^2$, using your estimated parameters (see Step A).
* D: Replace `xxxx` with the proper definition of the K-day holding return.
* E: Obtain the K-day Value-at-Risk forecast using the sample percentile: $VaR^p_{T+1:T+K} = -Percentile\bigl\{\{\check{R}_{i,T+1:T+K}\}_{i=1}^{MC},100p\bigr\}$, with $\{\check{R}_{i,T+1:T+K}\}_{i=1}^{MC}$ the simulated $K$-day returns. (Hint: use the `numpy` function `np.percentile`.)

Plot the histogram of $\{\check{R}_{i,T+1:T+K}\}_{i=1}^{MC}$, and estimate the skewness and excess kurtosis of the sample. Compare the MCS estimate of the 10-day VaR, with the 'square-root rule' VaR, $\sqrt{K} \, VaR^p_{T+1}$. Discuss your results. Can you give a reason why you find the specific ranking of VaR values?


In [ ]:
MC = 10000 # amount of Monte Carlo replications
VaR_level = 0.01 # the Value-at-Risk parameter `p`

K= 10 # The parameter K determins the length of the holding period, so we obtain a K-day return

# Obtain the GARCH parameters from `x`, the fitted model object generated by the .fit() method in the previous question
#---! STEP A:
[mu,omega,alpha,beta,d1] = x.params

# initialize a (MC x 1) vector to store the cumulative returns generated in the MC Monte Carlo replications
sim_cumulative_returns = np.zeros([MC,1])

# simulate the shocks 
sim_shocks = np.sqrt((d1 - 2) / d1) * stats.t.rvs(d1, size=[K,MC]) 

for i in range(MC): # we enter the `for loop' of Monte Carlo replications. In each iteration we compute a 10-day ahead return
    
    # initialize a (K x 1) return vector which will be filled with simulated returns for dates t+1,...,t+K (the vector is refreshed every Monte Carlo iteration)
    sim_returns = np.zeros([K,1])
    
    # initialize a (K x 1) conditional variance vector (the vector is refreshed every Monte Carlo iteration)
    sim_condvariance = np.zeros([K,1])
    
    for k in range(K): # we now generate K consecutive returns by iterating the volatility process, and utilizing the random shock draws
        
        if k == 0: # note that Python indexes the first element in a loop or vector as '0'
            # on the first day of the holding period we can use the 1-day ahead GARCH volatility forecast
            #---! STEP B:
            sim_condvariance[k] = xx
        elif k > 0: # for further days in the future we use the garch equation putting in the simulated shocks
            #---! STEP C:
            sim_condvariance[k] = xxx
        
        # calculate and save the simulated return
        sim_returns[k] = mu + np.sqrt(sim_condvariance[k]) * sim_shocks[k,i];    
        
    # estimate cumulative holding period return and save it as element 'i' in the (MC x 1) vector of sim_cumulative_returns
    #---! STEP D: 
    sim_cumulative_returns[i] = xxxx
            
# now we have simulated the cumulative returns for each of our Monte Carlo iterations we can estimate the Value-at-Risk using the sample percentile function
#---! Step E:
VaR = xxxxx

Discussion of results: 

## Question 3: Copula [15 pts.]

Suppose we have estimated another GARCH model for the FTSE index returns. Once we have the shocks of both assets, we estimate a bivariate normal copula model, and find an estimate for the copula correlation parameter, $\hat{\rho} = 0.80$. We want to simulate standardized $t$-shocks for the AEX and FTSE index returns, with the dependence between shocks modelled by this normal copula.

Below we have included code that performs simulation from the normal copula. 

Explain in detail what steps A to D achieve. Also explain how the scatter plot created at the end of the code shows the difference in correlation between the two sets of shocks, as well as the difference in fat-tailedness.



In [ ]:
# simulate the copula shocks

copula_corr = 0.80 # the parameter rho
d_1 = d1; # degrees of freedom parameter for standardized t distribition of AEX shock 
d_2 = d1; # degrees of freedom parameter for standardized t distribition of FTSE shock

#---! STEP A:
sim_uncorrelated_normal_shocks = stats.norm.rvs(size=[K,MC,2]) # result is a (K x MC x 2) array

#---! STEP B:
sim_correlated_normal_shocks = np.zeros([K,MC,2])
sim_correlated_normal_shocks[:,:,0] = sim_uncorrelated_normal_shocks[:,:,0]
sim_correlated_normal_shocks[:,:,1] = copula_corr * sim_uncorrelated_normal_shocks[:,:,0] + np.sqrt(1 - copula_corr**2) * sim_uncorrelated_normal_shocks[:,:,1] 

#---! STEP C:
sim_normal_copula_shocks = stats.norm.cdf(sim_correlated_normal_shocks)

#---! STEP D: 
sim_new_shocks = np.zeros([K,MC,2])
sim_new_shocks[:,:,0] = np.sqrt((d_1 - 2) / d_1) * stats.t.ppf(sim_normal_copula_shocks[:,:,0],d_1) 
sim_new_shocks[:,:,1] = np.sqrt((d_2 - 2) / d_2) * stats.t.ppf(sim_normal_copula_shocks[:,:,1],d_2) 

# create a scatter plot for MC shock vector draws
plt.title('Figure 4: Scatter plot of uncorrelated normal vector draws')
plt.scatter(sim_uncorrelated_normal_shocks[1,:,0], sim_uncorrelated_normal_shocks[1,:,1], alpha=0.5)

plt.title('Figure 5: Scatter plot of final shock vector')
plt.scatter(sim_new_shocks[1,:,0], sim_new_shocks[1,:,1], alpha=0.5)

Discussion of results:

#### Question 4: MCS of an equal-weighted portfolio return [ 10 pts.]

Adapt your MCS with the simulation scheme for the copula shocks above to obtain 10-day VaR forecasts for an equal-weighted portfolio in the AEX and FTSE indices, i.e. we create a portfolio of both assets with portfolio weights $w_1=w_2=0.5$. To circumvent further estimation, you may assume that the GARCH model parameters (including the degrees of freedom) for the AEX and FTSE returns are equal, that the conditional volatility $\sigma_{t+1}$ is equal between assets, and take copula parameter $\rho = 0.8$. 

How do your VaR estimates change when $\rho$ is decreased? Given what you know of log returns and simple returns, how would you compute the simple portfolio returns `sim_cumulative_returns[i]` if our index returns, `R_AEX` and `R_FTSE`, are log returns?

Discussion of results: